In [3]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.path.abspath('')).resolve().parents[0]))

In [ ]:
from information_noise_reduction.subset_generator import reverse_all_subsets_generator
from information_noise_reduction.evaluate_model import evaluate_subsets
from information_noise_reduction.interpretation import compute_variable_contributions

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

2024-11-09 14:37:51.946956: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load / Transform

In [6]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = (iris.target == 1).astype(int)  # Binary target for simplicity
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0
146,6.3,2.5,5.0,1.9,0
147,6.5,3.0,5.2,2.0,0
148,6.2,3.4,5.4,2.3,0


## Class of models

In [7]:
def model_generator(input_dim):
    model = Sequential([
            Dense(64, activation='relu', input_shape=(input_dim,)),
            Dense(1, activation='sigmoid')
        ])
    model.compile(optimizer='adam', loss='binary_crossentropy')

    return model

## Analysis

In [8]:
model_gen = model_generator
subset_gen = reverse_all_subsets_generator(df.columns[:-1].tolist())
result = evaluate_subsets(df, target_col='target', subset_gen=subset_gen, model_generator=model_gen, max_subsets=10, epochs=10)

2024-11-09 14:38:08.654548: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Evaluated subset ('sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)') with loss: 0.5526
Evaluated subset ('sepal length (cm)', 'sepal width (cm)', 'petal length (cm)') with loss: 0.6012
Evaluated subset ('sepal length (cm)', 'sepal width (cm)', 'petal width (cm)') with loss: 0.6331
Evaluated subset ('sepal length (cm)', 'petal length (cm)', 'petal width (cm)') with loss: 0.6195
Evaluated subset ('sepal width (cm)', 'petal length (cm)', 'petal width (cm)') with loss: 0.5626
Evaluated subset ('sepal length (cm)', 'sepal width (cm)') with loss: 0.6263
Evaluated subset ('sepal length (cm)', 'petal length (cm)') with loss: 0.6188
Evaluated subset ('sepal length (cm)', 'petal width (cm)') with loss: 0.6268
Evaluated subset ('sepal width (cm)', 'petal length (cm)') with loss: 0.5702
Evaluated subset ('sepal width (cm)', 'petal width (cm)') with loss: 0.5859


In [9]:
variable_contributions = compute_variable_contributions(result)
variable_contributions

{'average_losses': {'sepal length (cm)': 0.6111835667065212,
  'sepal width (cm)': 0.5902607526097979,
  'petal length (cm)': 0.5874720712502798,
  'petal width (cm)': 0.5967447459697723},
 'normalized_scores': {'sepal length (cm)': 0.2721335406622979,
  'sepal width (cm)': 0.5319488351703003,
  'petal length (cm)': 0.5665781190401817,
  'petal width (cm)': 0.45143191060069787}}